# Práctica de planificación automática

### Inteligencia Artificial
### Grado en Ingeniería Informática - Ingeniería del Software
### Universidad de Sevilla

Para implementar un problema de planificación se puede hacer uso de las clases
de objetos proporcionadas por el módulo `problema_planificación_pddl`
(**Nota**: es importante tener en cuenta que este módulo asume que todos los
símbolos de objetos son cadenas).

In [1]:
import problema_planificación_pddl as probpl

## Problema de la rueda pinchada

El problema de la rueda pinchada consiste en determinar los pasos a realizar
para cambiar una rueda pinchada por una rueda de repuesto que se encuentra en
el maletero, guardando finalmente la rueda pinchada en el maletero, para poder
continuar el viaje.

Cada predicado que se use para representar el problema debe crearse como una
instancia de la clase `Predicado`, indicando por separado el dominio de cada
variable (usando el tipo de dato `set` de Python para ello).

Para el problema de la rueda pinchada usaremos un predicado `EN(r, l)`, donde
`r` será una rueda (pinchada o de repuesto) y `l` una localización (eje,
maletero, suelo).

In [2]:
ruedas = {'rueda-pinchada', 'rueda-repuesto'}
localizaciones = {'eje', 'maletero', 'suelo'}
en = probpl.Predicado(ruedas, localizaciones)

Cada estado del problema es una instancia de la clase `Estado`, creada a partir
de instanciaciones de predicados previamente declarados.

In [3]:
estado_inicial_rueda = probpl.Estado(en('rueda-pinchada', 'eje'),
                                     en('rueda-repuesto', 'maletero'))
print(estado_inicial_rueda)

en(rueda-pinchada, eje)
en(rueda-repuesto, maletero)


Las acciones se implementan como instancias de la clase `AcciónPlanificación`.
Los argumentos que se pueden proporcionar son los siguientes:
* `nombre`: una cadena que representa la acción. Este argumento es obligatorio.
* `precondicionesP`: una lista de instanciaciones de predicados que forman las
  precondiciones positivas. Este argumento es opcional.
* `precondicionesN`: una lista de instanciaciones de predicados que forman las
  precondiciones negativas. Este argumento es opcional.
* `efectosP`: una lista de instanciaciones de predicados que forman los efectos
  positivos. Este argumento es opcional.
* `efectosN`: una lista de instanciaciones de predicados que forman los efectos
  negativos. Este argumento es opcional.
* `coste`: un número entero positivo (la implementación asume que el coste de
  aplicar la acción es siempre el mismo, independientemente del estado). Este
  argumento es opcional, en cuyo caso se toma coste 1.

En el caso de una sola precondición o efecto no es necesario proporcionarlos en
una lista.

In [4]:
# Sacar la rueda de repuesto del maletero
sacar = probpl.AcciónPlanificación(
    nombre='sacar_repuesto',
    precondicionesP=en('rueda-repuesto', 'maletero'),
    efectosP=en('rueda-repuesto', 'suelo'),
    efectosN=en('rueda-repuesto', 'maletero')
)

# Quitar la rueda pinchada del eje
quitar = probpl.AcciónPlanificación(
    nombre='quitar_pinchada',
    precondicionesP=en('rueda-pinchada', 'eje'),
    efectosP=en('rueda-pinchada', 'suelo'),
    efectosN=en('rueda-pinchada', 'eje')
)

# Colocar la rueda de repuesto en el eje
poner = probpl.AcciónPlanificación(
    nombre='poner_repuesto',
    precondicionesP=en('rueda-repuesto', 'suelo'),
    precondicionesN=en('rueda-pinchada', 'eje'),
    efectosP=en('rueda-repuesto', 'eje'),
    efectosN=en('rueda-repuesto', 'suelo')
)

# Guardar la rueda pinchada en el maletero
guardar = probpl.AcciónPlanificación(
    nombre='guardar_pinchada',
    precondicionesP=en('rueda-pinchada', 'suelo'),
    precondicionesN=en('rueda-repuesto', 'maletero'),
    efectosP=en('rueda-pinchada', 'maletero'),
    efectosN=en('rueda-pinchada', 'suelo')
)

Una vez creadas las acciones, la función `print` nos muestra su estructura.

In [5]:
print(quitar)


Acción: quitar_pinchada
  Precondiciones:
    en(rueda-pinchada, eje)
  Efectos:
    -en(rueda-pinchada, eje)
    en(rueda-pinchada, suelo)
  Coste: 1


In [6]:
print(guardar)


Acción: guardar_pinchada
  Precondiciones:
    -en(rueda-repuesto, maletero)
    en(rueda-pinchada, suelo)
  Efectos:
    -en(rueda-pinchada, suelo)
    en(rueda-pinchada, maletero)
  Coste: 1


Finalmente, un problema de planificación será una instancia de la clase
`ProblemaPlanificación` construida a partir de los siguientes argumentos:
* `operadores`: la lista de acciones del problema.
* `estado_inicial`: el estado inicial del problema.
* `objetivosP`: una lista de instancias de predicados que forman los objetivos
  positivos.
* `objetivosN`: una lista de instancias de predicados que forman los objetivos
  negativos.

En el caso de un solo operador, un solo objetivo positivo o un solo objetivo
negativo, no es necesario proporcionarlos en una lista.

In [7]:
problema_rueda_pinchada = probpl.ProblemaPlanificación(
    operadores=[quitar, guardar, sacar, poner],
    estado_inicial=estado_inicial_rueda,
    objetivosP=[en('rueda-pinchada', 'maletero'),
                en('rueda-repuesto', 'eje')]
)

Una vez implementado el problema de planificación, para buscar un plan solución
basta aplicar algún algoritmo de búsqueda en espacio de estados.

In [8]:
import búsqueda_espacio_estados as búsqee

In [9]:
búsqueda_profundidad = búsqee.BúsquedaEnProfundidad()
búsqueda_profundidad.buscar(problema_rueda_pinchada)

['sacar_repuesto', 'quitar_pinchada', 'poner_repuesto', 'guardar_pinchada']

In [10]:
búsqueda_anchura = búsqee.BúsquedaEnAnchura()
búsqueda_anchura.buscar(problema_rueda_pinchada)

['quitar_pinchada', 'sacar_repuesto', 'guardar_pinchada', 'poner_repuesto']

## Problema del mundo de los bloques

En el problema del mundo de los bloques se dispone de un conjunto de bloques
cúbicos dispuestos sobre una mesa. Los bloques se pueden apilar, pero cada
bloque solo se puede colocar sobre un único bloque. Un brazo robótico puede
coger un bloque y moverlo a otra posición, ya sea sobre la mesa o sobre otro
bloque. El brazo robótico solo puede coger un bloque cada vez. El objetivo es
construir una determinada pila de bloques.

En primer lugar declaramos los predicados que vamos a utilizar para representar
el problema, indicando el dominio de cada argumento. Si un predicado no tiene
argumentos, entonces se debe proporcionar el conjunto vacío como dominio.

In [11]:
bloques = {'A', 'B', 'C'}
despejado = probpl.Predicado(bloques)
brazolibre = probpl.Predicado({})
sobrelamesa = probpl.Predicado(bloques)
sobre = probpl.Predicado(bloques, bloques)
agarrado = probpl.Predicado(bloques)

Definimos un estado inicial para el problema de los bloques en el que: el bloque
A está situado sobre la mesa y no tiene nada encima; el bloque B está situado
sobre la mesa y tiene encima el bloque C; el bloque C no tiene nada encima; el
brazo robótico está libre.

In [12]:
estado_inicial_bloques = probpl.Estado(
    sobrelamesa('A'), despejado('A'),
    sobrelamesa('B'),
    sobre('C','B'), despejado('C'),
    brazolibre()
)

Se pueden establecer costes distintos para las acciones obtenidas a partir de
un mismo esquema. Para ello basta crear una instancia de la clase
`CosteEsquema` a partir de una función que establezca ese coste en función de
ciertos parámetros. Por ejemplo, supongamos que el coste de mover cada uno de
los tres bloques es distinto, ya que tienen pesos distintos.

In [13]:
coste_bloque = probpl.CosteEsquema(lambda b: {'A': 1, 'B': 2, 'C': 3}[b])

Los esquemas de acciones se implementan como instancias de la clase
`EsquemaPlanificación`. Los posibles argumentos que se pueden proporcionar son
los siguientes:
* `nombre`: una cadena de la forma $acc(z_{1}, \dotsc, z_{k})$, donde si $z_{i}$
  representa una variable, entonces debe escribirse entre llaves. Este
  argumento es obligatorio.
* `precondicionesP`: una lista de instancias de predicados que forman las
  precondiciones positivas. Este argumento es opcional.
* `precondicionesN`: una lista de instancias de predicados que forman las
  precondiciones negativas. Este argumento es opcional.
* `efectosP`: una lista de instancias de predicados que forman los efectos
  positivos. Este argumento es opcional.
* `efectosN`: una lista de instancias de predicados que forman los efectos
  negativos. Este argumento es opcional.
* `coste`: una instancia de la clase `costeEsquema` que establece el coste de
  una acción a partir de los valores de las variables $z_{i}$. Este
  argumento es opcional, en cuyo caso se toma coste 1.
* `dominio`: un conjunto de tuplas del mismo tamaño que el número de variables.
  Indica el conjunto de situaciones para las que tiene sentido instanciar el
  esquema de acción.
* `variables`: un diccionario que asocia a cada nombre de variable $z_{i}$ el
  conjunto de valores que puede tomar. 

Al menos uno de los argumentos `dominio` o `variables` debe aparecer. En caso
de incluir los dos, solo se tiene en cuenta el argumento `dominio`.

Las instancias de los predicados en `precondicionesP`, `precondicionesN`,
`efectosP` y `efectosN`, pueden hacer referencia a las variables $z_{i}$,
que deben escribirse entre llaves. En el caso de una sola precondición positiva
o negativa, o un solo efecto positivo o negativo no es necesario
proporcionarlos en una lista.

In [14]:
# El brazo robótico coloca un bloque sobre otro bloque
apilar = probpl.EsquemaPlanificación(
    nombre='apilar({x}, {y})',
    precondicionesP=[agarrado('{x}'),
                     despejado('{y}')],
    efectosN=[agarrado('{x}'),
              despejado('{y}')],
    efectosP=[brazolibre(),
              sobre('{x}', '{y}'),
              despejado('{x}')],
    coste=coste_bloque('{x}'),
    dominio={(x, y) for x in bloques for y in bloques if x != y}
)

# El brazo robótico coge un bloque que está sobre otro bloque
desapilar = probpl.EsquemaPlanificación(
    nombre='desapilar({x}, {y})',
    precondicionesP=[brazolibre(),
                     sobre('{x}', '{y}'),
                     despejado('{x}')],
    efectosN=[brazolibre(),
              sobre('{x}', '{y}'),
              despejado('{x}')],
    efectosP=[agarrado('{x}'),
              despejado('{y}')],
    coste=coste_bloque('{x}'),
    dominio={(x, y) for x in bloques for y in bloques if x != y}
)

# El brazo robótico coge un bloque que está sobre la mesa
agarrar = probpl.EsquemaPlanificación(
    nombre='agarrar({x})',
    precondicionesP=[brazolibre(),
                     sobrelamesa('{x}'),
                     despejado('{x}')],
    efectosN=[brazolibre(),
              sobrelamesa('{x}'),
              despejado('{x}')],
    efectosP=agarrado('{x}'),
    coste=coste_bloque('{x}'),
    variables={'x': bloques}
)

# El brazo robótico deja un bloque sobre la mesa
bajar = probpl.EsquemaPlanificación(
    nombre='bajar({x})',
    precondicionesP=agarrado('{x}'),
    efectosN=agarrado('{x}'),
    efectosP=[brazolibre(),
              sobrelamesa('{x}'),
              despejado('{x}')],
    coste=coste_bloque('{x}'),
    variables={'x': bloques}
)

La representación como cadena de un esquema de acción muestra las acciones que se generarían a partir de él.

In [15]:
print(agarrar)

Operador: agarrar({x})
ACCIONES GENERADAS:

Acción: agarrar(C)
  Precondiciones:
    brazolibre()
    sobrelamesa(C)
    despejado(C)
  Efectos:
    -brazolibre()
    -sobrelamesa(C)
    -despejado(C)
    agarrado(C)
  Coste: 3

Acción: agarrar(A)
  Precondiciones:
    brazolibre()
    sobrelamesa(A)
    despejado(A)
  Efectos:
    -brazolibre()
    -sobrelamesa(A)
    -despejado(A)
    agarrado(A)
  Coste: 1

Acción: agarrar(B)
  Precondiciones:
    brazolibre()
    sobrelamesa(B)
    despejado(B)
  Efectos:
    -brazolibre()
    -sobrelamesa(B)
    -despejado(B)
    agarrado(B)
  Coste: 2


In [16]:
print(apilar)

Operador: apilar({x}, {y})
ACCIONES GENERADAS:

Acción: apilar(C, B)
  Precondiciones:
    agarrado(C)
    despejado(B)
  Efectos:
    -agarrado(C)
    -despejado(B)
    brazolibre()
    sobre(C, B)
    despejado(C)
  Coste: 3

Acción: apilar(A, C)
  Precondiciones:
    agarrado(A)
    despejado(C)
  Efectos:
    -agarrado(A)
    -despejado(C)
    brazolibre()
    sobre(A, C)
    despejado(A)
  Coste: 1

Acción: apilar(B, C)
  Precondiciones:
    agarrado(B)
    despejado(C)
  Efectos:
    -agarrado(B)
    -despejado(C)
    brazolibre()
    sobre(B, C)
    despejado(B)
  Coste: 2

Acción: apilar(B, A)
  Precondiciones:
    agarrado(B)
    despejado(A)
  Efectos:
    -agarrado(B)
    -despejado(A)
    brazolibre()
    sobre(B, A)
    despejado(B)
  Coste: 2

Acción: apilar(A, B)
  Precondiciones:
    agarrado(A)
    despejado(B)
  Efectos:
    -agarrado(A)
    -despejado(B)
    brazolibre()
    sobre(A, B)
    despejado(A)
  Coste: 1

Acción: apilar(C, A)
  Precondiciones:
    agarrado(

Finalmente, para representar el problema de planificación se pasa la lista de
esquemas de acción a la clase `ProblemaPlanificación` (en general, se pueden
proporcionar tanto acciones como operadores, incluso mezclados).

In [17]:
problema_mundo_bloques = probpl.ProblemaPlanificación(
    operadores=[apilar,
                desapilar,
                agarrar,
                bajar],
    estado_inicial=estado_inicial_bloques,
    objetivosP=[sobrelamesa('C'),
                sobre('B', 'C'),
                sobre('A', 'B')]
)

Una vez implementado el problema de planificación, para buscar un plan solución
basta aplicar algún algoritmo de búsqueda en espacio de estados.

In [18]:
print(f'Estado inicial:\n{estado_inicial_bloques}')
print(f'Objetivos positivos: {problema_mundo_bloques.objetivosP}')
print(f'Objetivos negativos: {problema_mundo_bloques.objetivosN}')
búsqueda_profundidad.buscar(problema_mundo_bloques)

Estado inicial:
sobrelamesa(A)
sobrelamesa(B)
despejado(A)
despejado(C)
sobre(C, B)
brazolibre()
Objetivos positivos: {'sobrelamesa': {('C',)}, 'sobre': {('B', 'C'), ('A', 'B')}}
Objetivos negativos: {}


['desapilar(C, B)',
 'bajar(C)',
 'agarrar(B)',
 'apilar(B, C)',
 'agarrar(A)',
 'apilar(A, B)']

# Problema de los buceadores

En el marco de la _Conferencia Internacional sobre Planificación Automática y
Planificación Temporal_ ([International Conference on Automated Planning and
Scheduling, ICAPS](http://www.icaps-conference.org/)) se celebra, con
periodicidad aproximadamente trienal, la _Competición Internacional de
Planificación_ (https://www.icaps-conference.org/competitions/.

Esta competición tiene diferentes objetivos: realizar una comparación empírica
del estado del arte de los sistemas de planificación; destacar desafíos para la
comunidad de Planificación Automática; proponer nuevas direcciones para la
investigación y nuevos vínculos con otros campos de la Inteligencia Artificial;
y proporcionar nuevos conjuntos de datos que puedan ser utilizados por la
comunidad científica como puntos de referencia.

Uno de los problemas incluidos en la competición es el _problema de los
buceadores_, propuesto por Nathan Robinson, Christian Muise y Charles Gretton.

El problema consiste en lo siguiente: hay una serie de buceadores, cada uno de
los cuales puede acarrear 4 tanques de aire. A estos buceadores hay que
contratarlos para que entren en un sistema cavernoso inundado y, o bien tomen
fotografías, o bien preparen el camino para otros buceadores dejando caer
tanques llenos de aire. El lugar es demasiado estrecho para que pueda entrar
más de un buceador a la vez. El sistema cavernoso está formado por una serie de
cuevas, algunas de ellas interconectadas entre sí. La entrada es única. Ciertas
cuevas son objetivos que los buceadores deben fotografiar. Tanto nadar de un
lugar a otro, como fotografiar una cueva, consume un tanque entero de aire. Los
buceadores deben realizar un proceso de descompresión al salir a superficie,
por lo que cada uno de ellos solo puede realizar un único viaje. Ciertos
buceadores desconfían de algunos de sus compañeros y rechazarán trabajar si
alguno de ellos ha recorrido las cuevas previamente. Contratar un buceador
tiene un coste diferente para cada uno de ellos.

Consideremos los siguientes conjuntos de símbolos de objetos (**que no tienen
por qué ser los únicos que se usen en el problema**):

In [19]:
cuevas = {f'C{i}' for i in range(5)}
buceadores = {f'B{i}' for i in range(2)}
cantidades = {str(i) for i in range(9)}

In [20]:
print("Cuevas: {}".format(cuevas))
print("Buceadores: {}".format(buceadores))
print("Cantidades: {}".format(cantidades))

Cuevas: {'C0', 'C2', 'C1', 'C4', 'C3'}
Buceadores: {'B1', 'B0'}
Cantidades: {'2', '5', '0', '7', '3', '8', '4', '1', '6'}


y las siguientes relaciones de conexión entre las cuevas:

In [21]:
conexiones = [('C0', 'C1'),
              ('C1', 'C0'),
              ('C1', 'C2'),
              ('C1', 'C4'),
              ('C2', 'C1'),
              ('C2', 'C3'),
              ('C3', 'C2'),
              ('C4', 'C1')]

**Ejercicio 1**: implementar los siguientes predicados:
* `posición_buceador`: para cada buceador indica en qué cueva se encuentra, o
  si se encuentra en la superficie.
* `disponible`: para cada buceador indica si está disponible para trabajar.
* `trabajando`: para cada buceador indica si está contratado y trabajando.
* `descompresión`: para cada buceador indica si está en el proceso de
  descompresión.
* `tanques_llenos`: para cada buceador indica cuantos de sus 4 tanques están
  llenos de aire; para cada cueva indica cuantos tanques llenos de aire hay en
  dicha cueva, para que un buceador los pueda coger.
* `con_foto_de`: para cada cueva indica si se le ha realizado o no una
  fotografía.

**Ejercicio 2**: implementar las siguientes acciones:
* `contratar(B0)`: contrata al buceador `B0`, que inmediatamente se dispone a
  trabajar; siempre y cuando esté disponible y no haya otro buceador contratado
  ahora mismo. El buceador `B1` rechazará ser contratado después de él.
  Contratar al buceador `B0` tiene coste 10.
* `contratar(B1)`: contrata al buceador `B1`, que inmediatamente se dispone a
  trabajar; siempre y cuando esté disponible y no haya otro buceador contratado
  ahora mismo. Contratar al buceador `B1` tiene coste 67.

**Ejercicio 3**: implementar los siguientes operadores:
* `entrar_al_agua`: un buceador contratado entra desde la superficie al sistema
  cavernoso, lleva sus cuatro tanques de aire llenos.
* `bucear`: un buceador nada entre dos cuevas conectadas, gastando un tanque
  completo de aire.
* `fotografiar`: un buceador fotografía una cueva, gastando un tanque completo
  de aire.
* `soltar_tanque`: un buceador suelta un tanque lleno en una de las cuevas.
* `cargar_tanque`: un buceador carga uno de sus tanques vacíos con uno lleno
  que se ha soltado previamente en una de las cuevas.
* `salir_del_agua`: un buceador sale a superficie y pasa al proceso de
  descompresión.

**Ejercicio 4**: implementar la instancia del problema de tal manera que
inicialmente los dos buceadores estén en la superficie, disponibles para ser
contratados; no haya tanques llenos en las cuevas; y no se haya hecho todavía
ninguna foto. El objetivo será fotografiar la cueva `C4` y que los dos
buceadores estén en la superficie.

**Ejercicio 5**: Aplicar algún algoritmo de búsqueda en espacio de estados para
encontrar un plan solución de la instancia del problema (**Nota**: una búsqueda
no informada puede requerir un tiempo considerable). ¿Cuántas acciones tiene el
plan resultante?. ¿Se puede alcanzar el mismo objetivo pero con una foto de la
cueva `C3`?